In [1]:
from pyspark.sql import Row
from pyspark.sql.types import *
import ast
import json
import pandas as pd
import numpy as np
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import HiveContext
hiveContext=HiveContext(sc)

In [2]:
#dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

In [3]:
#The below code needs to be run on an ec2 machine after downloading the cv files. 
#The code generates processed json files and pushes it to s3

import pandas as pd
import json
import ast
import requests
import pickle
import os

def loadUpdatedTags():
	url="https://s3-us-west-2.amazonaws.com/data-analysis-sachin/tags.json"
	tagsResponse=requests.get(url)
	j=json.loads(tagsResponse.text)
	dfs=[]
	keys=j.keys()
	ddict={}
	for key in keys:
		template=[]
		for d in j[key]['contexts']:
			if(d['keyword']=='room'):
				room=d['tag']
		for d in j[key]['sensorpoints']:
			template.append(d['description'])
		ddict[str(key)]={'sensor_id':str(key),'room':room,'template':template}
	return ddict

sensorRoomDict=loadUpdatedTags()
#print sensorRoomDict['876d4cd6-7808-11e3-9b2e-00163e005319']
def csvParser(path):
    #print "starting ",path
    dfs={}
    with open(path) as f:
        l=f.readlines()
    print "lines=",len(l)
    i=0
    while(i<len(l)):
        try:
            #print path[:-4]
            df=pd.DataFrame(columns=["sensor_id","timeseries","values","room","template"])
            #df["sensor_value"]=l[i].strip()
            df["timeseries"]=l[i+1].split(',')
            df["values"]=l[i+2].split(',')
            df["sensor_id"]=path[:-4]
            try:
                df["room"]=sensorRoomDict[path[:-4]]['room']
            except:
                df["room"]="not_found"
            try:
            	df['template']=sensorRoomDict[path[:-4]]['template'][i/3]
            except:
                df["template"]="not_found"
            dfs[path[:-4]]=df
            i+=3
        except:
            print "parser exception ",i/3
            i+=3
    return dfs

#write loop to read csv and push dfs to s3
directory=os.getcwd()
paths=os.listdir(directory)
j=0
for i in xrange(len(paths)):
    if(i<0):
        continue
    else:
        try:
            #print "starting ",i,paths[i]
            dfDict=csvParser(paths[i])
            for k,v in dfDict.iteritems():
                f="DF_"+str(k)+".json"
                print f
                v.to_json(f,orient="split")
                url="s3cmd put --acl-public --guess-mime-type "+f+" s3://data-analysis-sachin/csv2jsons_new/"+f
                os.system(url)
                cmd="rm "+f
                os.system(cmd)
                j+=1
        except:
            print "exception in ",i,paths[i]



def jsonParser(line):
    dictRow={}
    row=line.split("}}")
    i=0
    dfs=[]
    while i<len(row):
        try:
            r,t=row[i].split('timeseries":')
            splits=r.split('"')
            timeValues=json.loads(t)
            timeValuesDict={}
            for tv in timeValues:
                timeValuesDict[tv.keys()[0]]=tv.values()[0]
            tv_df=pd.DataFrame.from_dict(timeValuesDict,orient='index')
            if(len(splits)>=2):
                room=""
                if splits[1] in sensorRoomDict.keys():
                    room=sensorRoomDict[splits[1]]
                tv_df.columns=["values"]
                print splits[3]
                tv_df['sensor_value']=splits[3]
                tv_df['room']=sensorRoomDict[splits[1]]
                tv_df['sensor_id']=splits[1]
                dfs.append(tv_df)
        except:
            print "inside file exception - row"+str(i)
        i=i+1
    if(len(dfs)>0):
        combined_df=pd.concat(dfs)
        return combined_df
    else:
         return "no dataframe"  

In [4]:
#Code to read jsons from s3 and save them as parquet files
def readData(m,n):
  pd_df_dict={}
  for i in xrange(m,n):
      try:
        if(i%100==0):
          print i
        pd_df_dict[i]=pd.read_json("s3://data-analysis-sachin/csv2jsons_new/DF_"+str(i)+".json",orient='split')
      except:
        print "exception", i
  return pd_df_dict

k=0
for i in xrange(0,16440,20):
  j=i+20
  pd_df_dict=readData(i,j)
  pd_all_concat=pd.concat(pd_df_dict.values())
  sp_df_all=sqlContext.createDataFrame(pd_all_concat)
  path="/mnt/assignment1/csvParquetFiles_new/processed_"+str(k)
  sp_df_all.write.parquet(path)
  k+=1

In [6]:
#Old/outdated analysis/code below

In [7]:
timeCorrected_DF=timeCorrected_DF.withColumn('TimeStamp', from_unixtime('TimeStamp', format='yyyy-MM-dd HH:mm:ss'))

In [8]:
timeCorrected_DF.saveAsTable("batch1_buildingSensorAnalysis_timeCorrected")

In [9]:
timeCorrected_DF=sqlContext.table('batch1_buildingSensorAnalysis_timeCorrected')

In [10]:
# Time Slicing #
timeSliced_DF=timeCorrected_DF[timeCorrected_DF['TimeStamp']<='2013-12-27 17:30:00']
timeSliced_DF=timeSliced_DF[timeSliced_DF['TimeStamp']>='2013-12-27 09:00:00']

In [11]:
timeSliced_DF.count()

In [12]:
timeSliced_DF.columns

In [13]:
RoomGrouped_DF=timeSliced_DF.select('Room','Actual_Cooling_Setpoint',
 'Actual_Heating_Setpoint','Temperature','Zone_Temperature').groupBy('Room')

In [14]:
RoomGrouped_DF.avg().show() #RoomWise average for the sensors on the day 27th Dec 8am-5.30pm

In [15]:
### Visualizations ###
display(timeSliced_DF)

In [16]:
display(timeSliced_DF)

In [17]:
display(timeSliced_DF)

In [18]:
display(timeSliced_DF)

In [19]:
display(timeSliced_DF)

In [20]:
def dataPlot(dfArgument,timeLowerBound, timUpperBound, Rooms):
  tempDF=dfArgument[dfArgument.TimeStamp.between(timeLowerBound, timUpperBound)]
  if(Rooms!='All'):
    tempDF=tempDF[tempDF['room']==Rooms]    
  display(tempDF)
  return tempDF

In [21]:
timeSlicedRoomFilteredDF=dataPlot(timeCorrected_DF,'2014-04-23 08:00:00', '2014-04-30 17:30:00',4126)

In [22]:
display(timeSlicedRoomFilteredDF.withColumn('Actual_Supply_Flow',timeSlicedRoomFilteredDF.Actual_Supply_Flow/20.+50))

In [23]:
#Reading Parquet Files

In [24]:
from pyspark.sql import HiveContext
hiveContext=HiveContext(sc)

In [25]:
dfs={}
for i in xrange(19):
  path="/mnt/assignment1/parquetFiles/processed_"+str(i)
  dfs[i]=hiveContext.read.parquet(path)
  print dfs[i].select('sensor_value').distinct().collect()

In [26]:
df=dfs[0]
for i in xrange(1,19):
  df.unionAll(dfs[i])

In [27]:
from datetime import datetime
df2=df.map(lambda row: Row(**dict(row.asDict(), TimeStamp=datetime.strptime(row.TimeStamp,"%d-%m-%y %H:%M"))))
df3=sqlContext.createDataFrame(df2)
df3.describe

In [28]:
df3.saveAsTable('rd0_19_del_others')

In [29]:
df=df3
df.count()

In [30]:
from pyspark.sql import functions as F
print df.agg(F.min(df.TimeStamp)).collect()
df.agg(F.max(df.TimeStamp)).collect()

In [31]:
df.groupBy('room','sensor_id').count().collect()

In [32]:
df.select('room').distinct().collect()

In [33]:
dfb103A=df.where(df.room=='RM-B103A')

In [34]:
df2=df.map(lambda row: Row(**dict(row.asDict(), date=datetime.date(row.TimeStamp))))
df=sqlContext.createDataFrame(df2)

In [35]:
dfPlot=dataPlot(df,'2014-01-01 08:00:00', '2014-12-30 17:30:00', 'RM-B103A')

In [36]:
display(dfPlot)

In [37]:
#Getting dfPlot in pandas
days=dfPlot.select('date').distinct().collect()
days=[d.date for d in days]
print dfPlot.columns
pdPlot=pd.DataFrame(dfPlot.collect(),columns=['TimeStamp','date','room','sensor_id','sensor_value','values'])
temp=pdPlot['TimeStamp']
t=temp.map(lambda x:datetime.time(x))
pdPlot['dayTime']=t

In [38]:
#Plotting Pandas DF
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for day in days[0:5]:
  pdSubPlot=pdPlot[pdPlot['date']==day]
  print pdSubPlot.columns
  pdSubPlot=pdSubPlot.sort(['dayTime'],ascending=True)
  print pdSubPlot[0:4]
  plt.plot(pdSubPlot['dayTime'],pdSubPlot['values'])

In [39]:
display(fig)

In [40]:
#Encder series based on pandas DF. To be modified for Spark

In [41]:
pdPlot.columns

In [42]:
import sys
A=pdPlot
print 'shape before dropping Room==nan =',shape(A)
A=A.dropna(subset=['room'])
print 'shape after dropping Room==nan =',shape(A)
rooms=sorted(list(set(A.room.values)))

print 'starting Piecewise, rooms=',rooms
sys.stdout.flush()

cols = list(A.columns)
cols.remove('room')
#cols.remove('Temperature')


In [43]:
#import numpy as np
shape(A['values'])

In [44]:
room=rooms[0]
print '-'*20,room,'-'*20
sys.stdout.flush()
DF = A[A['room']==room]
DF = DF.drop('room', 1)  # remove the entry "Room"
name='values'
S=DF[name][0:200]
_std=std(S)

In [45]:
from numpy import *
print 'room=',room,'signal=',name, 'std=',_std,
encoder=piecewise_constant(S)


In [46]:
C=encoder.compress()
print 'size=',encoder.get_size(),
error=encoder.compute_error(S,compressed=C)
print 'error=',error, 'error/_std=',error/_std

In [47]:
C

In [52]:
### Old code below ###

In [53]:
pd_df1=pd.read_json("s3://data-analysis-sachin/jsonfiles/150days_split"+str(1)+".json")

In [54]:
pd_df_concat=pd.concat([pd_df0,pd_df1])

In [55]:
pd_df_concat.shape

In [56]:
sp_df=sqlContext.createDataFrame(pd_df_concat)
sp_df.head(5)

In [57]:
sp_df.saveAsTable("test")

In [58]:
sp_df.write.parquet("/mnt/assignment1/test.parquet")

In [59]:
dbutils.fs.ls('/mnt/assignment1/test.parquet/')

In [60]:
rawdata_009=sc.textFile("/mnt/assignment1/rawdata_009.json")

In [61]:
rd009=rawdata_009.map(lambda r:r.split('PresentValue'))

In [62]:
rd009.collect()

In [63]:
import ast
#rawdata_047 = sc.textFile("/mnt/assignment1/rawdata_047.json")
#rd047=rawdata_047.take(1)
#testdf=pd.read_json(rd047[0])
testdf[u'2f1dbf56-8917-11e2-b769-00163e005319']

In [64]:
rawdata_047.

In [65]:
import numpy as np
import math
AllData={}
for i in xrange(10):
  #print a[a.columns[i]][u'Test SensorPoint']
  tag=tags[c.columns[i]]['contexts']
  for t in tag:
    if(t['keyword']=='room'):
      room=str(t['tag'])
      roomSensors={}
  for k in c[c.columns[i]].keys():
    if(type(c[c.columns[i]][k])==dict):
      if('timeseries' in c[c.columns[i]][k].keys()):
        time={}
        for d in c[c.columns[i]][k]['timeseries']:
          time[d.keys()[0]]=d.values()[0]
      if(k in roomSensors.keys()):
        roomSensors[k].update(time)
      else:
        roomSensors[k]=time
  
  sensorTimeDF=pd.DataFrame.from_dict(roomSensors,orient='index')
  #timeDF.columns=['time']
  if(room in AllData.keys()):
    oldDf=AllData[room]
    AllData[room]=pd.concat([oldDF,sensorTimeDF])
  else:
    AllData[room]=sensorTimeDF
  
print AllData['RM-1109']
  #AllData[room]
  

In [66]:
files={f1:pd.read_json("s3://data-analysis-sachin/Copy of rawdata_054.json"),f2:pd.read_json("s3://data-analysis-sachin/Copy of rawdata_055.json"),f3:pd.read_json("s3://data-analysis-sachin/Copy of rawdata_056.json"),f4:pd.read_json("s3://data-analysis-sachin/Copy of rawdata_057.json"),f5:pd.read_json("s3://data-analysis-sachin/Copy of rawdata_058.json")}

In [68]:
import numpy as np
import pandas as pd
tags=pd.read_json("s3://data-analysis-sachin/tags.json")
import math
fileDataDict={}
fileDataDF={}
for fileNum in xrange(54,59):
  f=pd.read_json("s3://data-analysis-sachin/Copy of rawdata_0"+str(fileNum)+".json")
  AllSensorData={}
  for i in xrange(len(f.columns)):
    tag=tags[f.columns[i]]['contexts']
    roomSensor={}
    for t in tag:
      if(t['keyword']=='room'):
        room=str(t['tag'])
        print room
    for k in f[f.columns[i]].keys():
      #df=pd.DataFrame([[0,0,0]],columns=['time',k,'Room'])
      if(type(f[f.columns[i]][k])==dict):
        if('timeseries' in f[f.columns[i]][k].keys()):
          values=[]
          for d in f[f.columns[i]][k]['timeseries']:
            values.append([d.keys()[0],d.values()[0],room])
          
          df=pd.DataFrame(values,columns=['time',k,'Room'])
          if(room not in roomSensor.keys()):
            roomSensor[room]=df
          else:
            old_df=roomSensor[room]
            roomSensor[room]=pd.merge(old_df,df,on=['time','Room'],how='right')
    AllSensorData[i]=pd.concat(roomSensor.values())
  fileDataDF[f]=pd.concat(AllSensorData.values())
  fileDataDict[f]=AllSensorData[i]


print fileDataDF.head()


In [69]:
AllData['RM-1108'].columns

In [70]:
df=pd.DataFrame([[0,0,0]],columns=['time','k','Room'])
df = pd.DataFrame([[1, 2], [3, 4]])
print df
#df.append([[5,6]])
df2=pd.DataFrame([[5,6]])
print df2
dictodf={}
dictodf['a']=df
dictodf['b']=df2
df3=pd.concat(dictodf.values())
print df3

In [71]:
import numpy as np
import math
AllData={}
for i in xrange(10):
  #print a[a.columns[i]][u'Test SensorPoint']
  tag=tags[c.columns[i]]['contexts']
  for t in tag:
    if(t['keyword']=='room'):
      room=str(t['tag'])
      if(room in AllData.keys()):
        roomSensors=AllData[room]
      else:
          roomSensors={}
  for k in c[c.columns[i]].keys():
    if(type(c[c.columns[i]][k])==dict):
      if('timeseries' in c[c.columns[i]][k].keys()):
        time={}
        for d in c[c.columns[i]][k]['timeseries']:
          time[d.keys()[0]]=d.values()[0]
      timeDF=pd.DataFrame.from_dict(time,orient='index')
      if(k in roomSensors.keys()):
        roomSensors[k].concat(timeDF)
      else:
        roomSensors[k]=timeDF
  if(room in AllData.keys()):
    AllData[room].update(roomSensors)
  else:
    AllData[room]=roomSensors
  
print AllData
  #AllData[room]

#for k in c[c.columns[0]].keys():
#   if(type(c[c.columns[0]][k])==dict):
#      if('timeseries' in c[c.columns[0]][k].keys()):
#        print len(c[c.columns[0]][k]['timeseries'])
#for i in range(0,9):
  #print len(c[c.columns[0]].to_dict().values()[0]['timeseries'])
#  print type(a[a.columns[i]]['Test SensorPoint'])
  #print a[a.columns[i]].to_dict()


In [72]:
for i in xrange(54,58):
  print "s3://data-analysis-sachin/Copy of rawdata_0"+str(i)+".json"